In [18]:
import pandas as pd
import numpy as np
from google.colab import drive
import os

In [19]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
filenames = ["albany,ny.csv", "boston,ma.csv", "chicago,il.csv", "san+francisco,ca.csv", "springfield,il.csv", "springfield,ma.csv", "los+angeles,ca.csv", "new+york,ny.csv"]
dataframes = {}

In [21]:
for i, f in enumerate(filenames):
    without_extension = os.path.splitext(f)[0]
    splitted = without_extension.split(",")
    df_name = f"df{i}"
    df = pd.read_csv(f"/content/drive/MyDrive/data/{f}", usecols=[0, 1, 2, 19], names=['date_time', 'max_temp', 'min_temp', 'precipMM'], header=0)
    df["city"] = splitted[0]
    df["state"] = splitted[1]
    dataframes[df_name] = df

In [22]:
df_all = pd.DataFrame()
for df in dataframes.values():
    df_all = pd.concat([df_all, df])

In [23]:
df_all.head()

,date_time,max_temp,min_temp,precipMM,city,state
0,2018-12-11 00:00:00,-2,-8,0.0,albany,ny
1,2018-12-11 03:00:00,-2,-8,0.0,albany,ny
2,2018-12-11 06:00:00,-2,-8,0.0,albany,ny
3,2018-12-11 09:00:00,-2,-8,0.0,albany,ny
4,2018-12-11 12:00:00,-2,-8,0.0,albany,ny


In [45]:
def has_multiple_readings_at_least(mini_df, min_mm, times):
  return mini_df.loc[mini_df['precipMM'] >= min_mm, 'precipMM'].count() >= times

In [47]:
df_all.groupby(["city", "state"]).filter(has_multiple_readings_at_least, min_mm=15, times=4)[['city', 'state']].drop_duplicates()

,city,state
0,los+angeles,ca
0,new+york,ny


In [25]:
df_prec_more_than_15.head()

,date_time,max_temp,min_temp,precipMM,city,state
323,2019-01-20 09:00:00,4,-4,16.0,boston,ma
358,2019-01-24 18:00:00,11,9,20.2,boston,ma
359,2019-01-24 21:00:00,11,9,15.8,boston,ma
710,2019-03-09 18:00:00,3,-4,15.7,chicago,il
515,2019-02-13 09:00:00,14,11,23.7,san+francisco,ca


In [26]:
df_prec_more_than_15.groupby(["city", "state"]).count()

,,date_time,max_temp,min_temp,precipMM
city,state,,,,
boston,ma,3,3,3,3
chicago,il,1,1,1,1
los+angeles,ca,7,7,7,7
new+york,ny,4,4,4,4
san+francisco,ca,2,2,2,2


In [31]:
df_prec_more_than_10 = df_all.loc[df_all["precipMM"] >= 10]
df_temp_less_than_0 = df_prec_more_than_10.loc[df_prec_more_than_10["max_temp"] < 0]
df_temp_less_than_0

,date_time,max_temp,min_temp,precipMM,city,state
322,2019-01-20 06:00:00,-7,-10,10.7,albany,ny
323,2019-01-20 09:00:00,-7,-10,12.9,albany,ny
511,2019-02-12 21:00:00,-4,-10,11.2,albany,ny


In [33]:
df_all.groupby(["city", "state"])["precipMM"].sum()

city           state
albany         ny       448.2
boston         ma       418.2
chicago        il       274.2
los+angeles    ca       519.9
new+york       ny       505.9
san+francisco  ca       419.4
springfield    il       203.8
               ma       128.7
Name: precipMM, dtype: float64